In [ ]:
#Hello World
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import os


In [48]:
os.getcwd()

'/home/jahn42/music_ml'

In [49]:
df = pd.read_csv('tracks_features.csv', low_memory=False)

In [54]:
""" 
This dataset contains 1,204,025 tracks with numeric features describing the track. The data comes from Spotify :)
It has music up until 2020, so there will be a lot of new music that won't be in this dataset. If you 
guys find a more recent dataset, lmk. "

153,092 Artists

"""
df.sample(n=10, random_state=42)

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date,month
776583,1hU7uzX8LCE20HSjwVsjeq,Vert-Horiz,Three/Three,4lS4VoFPUEibJzxjzTeosc,['Dabrye'],['1tVRa7YiCwE6hTGsqqb4Te'],13,1,False,0.626,...,0.64700,0.955000,0.0745,0.462,98.015,164653,4.0,2018,2018-02-16,02
507443,4P714HDYnrwTBqNiVmVRyH,Sidewave,Instruments Of Mass Pleasure,1qXXGyFZErABvHSaciLX8n,['Alex Budman And The Contemporary Jazz Orches...,['5rifqIY150V8R4RbECUqng'],6,1,False,0.448,...,0.67700,0.679000,0.3730,0.503,123.387,444053,4.0,2006,2006-04-15,04
1197564,7nwbzU7pMXyBIRg7jMRwvB,En el Aire,Sudando Rock´n´roll,3PyQiXcvEMBURLFTSCWdRf,['La Pulposa'],['5SN0hsfwTuMP49GwxuVmjO'],8,1,False,0.604,...,0.15700,0.010600,0.1140,0.822,105.411,268566,4.0,2017,2017-08-21,08
745558,7bIx9uMIk8LceIynHiefiD,"Göttlichs Kind, lass mit, TWV 1:1020: I. Kindl...",Telemann: Christmas Cantatas,5RrZ2th6REPwOly3b192bs,"['Georg Philipp Telemann', 'Dirk Schmidt', 'Mi...","['2fg5h5pzleqI4RjFopWroj', '1tVkADvxz3pckJtXdI...",25,1,False,0.492,...,0.96000,0.000009,0.0580,0.568,120.497,93827,4.0,2000,2000-10-06,10
1127695,3lD3wzQOORpRYbzHarXlaM,Look What You Done for Me,Get Soulful with Al Jarreau,6F4QDi5eHSE5yospCR2O0U,['Al Jarreau'],['3YR92OLKlvkK5oKNekSqXe'],8,1,False,0.563,...,0.68100,0.000001,0.2970,0.827,97.143,160893,4.0,2013,2013-12-08,12
641979,18YeWePCq2QaXeul0pq694,魑魅魍魎,#10,5flhMOLgXfn5dB2zelTitY,"['FLOW', 'Hayate']","['3w2HqkKa6upwuXEULtGvnY', '2D7OUEgyRCd1Se4UL8...",6,1,False,0.557,...,0.00281,0.000003,0.2550,0.815,95.541,244333,4.0,2016,2016-02-03,02
1097810,1sTvJD1S3ERwJGL3TH6MEO,Wailing Rudie,Cool Ruler Come Again,5ZGSAEbxJ1FDyStbilrsMT,['Gregory Isaacs'],['6QHu71f8LLeT8n0GzfbYFc'],5,1,False,0.656,...,0.22500,0.000001,0.1070,0.722,73.428,229099,4.0,2017,2017-01-06,01
166380,64bZyPVZQsqTw6zcv1QzTQ,Live It Up,Fully Qualified,1ojZJ7HHJGMOuGLWnwfUsh,['Predator Dub Assassins'],['1QpFMjIJtID4dMO031sW5D'],12,1,False,0.794,...,0.00752,0.000000,0.0476,0.970,130.009,207800,4.0,2009,2009-12-16,12
580388,3ttSQIcg8rDr47jRUAvPQ8,I Try,Stand Up,3V03VFgV3X3FH5SWJd7aGD,['Superiority Complex'],['6xrZqC18N2SXGIHyxNgDXg'],10,1,False,0.474,...,0.33000,0.000000,0.4650,0.613,177.335,188373,4.0,2006,2006-04-10,04
796207,31X185MlGFhRbVTE9wuo5T,Yet Another Movie - Live,Delicate Sound of Thunder (Live),4Iy1JP7YE0d256eCwn3VJA,['Pink Floyd'],['0k17h0D3J5VfsdmQ1iZtE9'],3,1,False,0.281,...,0.01280,0.005350,0.4730,0.275,94.946,381173,4.0,1988,1988-11-22,11


In [55]:
df["month"] = df['release_date'].str.split("-").str[1]

In [56]:
"""
This function takes in the name of an artist and the name of the song as parameters
It searches the dataset and returns a dataframe with the following columns:

"year", "tempo", "time_signature", "energy", "danceability", "key", "instrumentalness"

These are the features I chose for KNN but we can change them later to optimize
"""
def find_artist_song(name, song):
    str_1= "['"
    str_2 = "']"
    new = str_1 + name + str_2
    temp = df[df['artists'] == new]
    temp = temp[temp['name'].str.contains(song)]
    return temp[["year", "tempo", "time_signature", "energy", "danceability", "key", "instrumentalness"]]

In [57]:
df = df.dropna() # drop missing values
df.columns

Index(['id', 'name', 'album', 'album_id', 'artists', 'artist_ids',
       'track_number', 'disc_number', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'year', 'release_date', 'month'],
      dtype='object')

In [58]:


# These are the features I chose before to find Y, the songs that are most similar to the input.
X = df[["year", "tempo", "time_signature", "energy", "danceability", "key", "instrumentalness"]] 




In [59]:
Y = df["name"] # name of track that is most similar

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42) #Train Test Split

In [61]:
knn_model = KNeighborsClassifier(n_neighbors=5) # I chose 5 so that we get 5 recommended songs

knn_model.fit(X_train, y_train)


KNeighborsClassifier()

In [62]:
y_pred = knn_model.predict(X_test)

# calculate the accuracy of the model.

accuracy = accuracy_score(y_test, y_pred)

print("Accuracy: {:.2f}%".format(accuracy * 100))


Accuracy: 0.77%


In [114]:
def recommend_songs(favorite_song, favorite_artist, df, knn_model, k=5):

    #filter data
    favorite_song_info = df[(df['name'].str.contains(favorite_song)) & (df['artists'].str.contains(favorite_artist))]
    
    if favorite_song_info.empty:
        return f"No matching song '{favorite_song}' by artist '{favorite_artist}' found in the dataset."
    
    #extract the feature vector for the favorite song
    favorite_song_features = favorite_song_info[["year", "tempo", "time_signature", "energy", "danceability", "key", "instrumentalness"]]
    
    # Use k-NN to find similar songs
    distances, indices = knn_model.kneighbors(favorite_song_features, n_neighbors=5)

    # get the recommended songs based on indices
    recommended_songs = df.iloc[indices[0]]
    
    # filter out the favorite song 
    recommended_songs = recommended_songs[recommended_songs['name'] != favorite_song]
    
    # return the song recommendations with artist names
    recommendations = recommended_songs[['name', 'artists','year']]
    return recommendations

# example usage:
favorite_song = "Yellow"  # replace with user input
favorite_artist = "Coldplay"  # replace with user input

recommendations = recommend_songs(favorite_song, favorite_artist, df, knn_model)
print(recommendations)
# The first 5 results are the name of the songs
# The next 5 results are the artist names + year 

                                                  name  \
236037                                  Jam It, Jam It   
292025                                     Competition   
603311                                            Burn   
57787   Here Comes the Sweater Weather (Piano Version)   
413831                                      Alien Life   

                                  artists  year  
236037                         ['Clay D']  1994  
292025             ['The Brain Surgeons']  1999  
603311  ['Star Anna & The Laughing Dogs']  2009  
57787                       ['Lullatone']  2019  
413831               ['Thirteenth Tribe']  2014  


In [113]:
def find_artist(name):
    str_1= "['"
    str_2 = "']"
    new = str_1 + name + str_2
    temp = df[df['artists'] == new]
    return temp